In [2]:
import pandas as pd                     
from pandas import DataFrame           
import numpy as np                      

from itertools import chain

from collections import Counter         
from datetime import datetime           
import re                              
from time import sleep                  
import random                           
import math                            
import time                             
import itertools                       
import sys
import openpyxl


import requests                        
from bs4 import BeautifulSoup           
from selenium import webdriver         
import json                            

In [15]:
def get_soup_elements(typ_obchodu = "prodej", typ_stavby = "byty", pages = 1):  
    
    browser = webdriver.Chrome()
    
    
    url_x = r"https://www.sreality.cz/hledani"             
    url = url_x + "/" +  typ_obchodu + "/" +  typ_stavby

    browser.get(url)    # (url).text ??
    sleep(random.uniform(1.0, 1.5))
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,'lxml') # "parser" ??
    
    elements = []    
    
    for link in soup.findAll('a', attrs={'href': re.compile("^/detail/")}):      
        link = link.get('href')   
        elements.append(link)     
    elements = elements[0::2]   

    
    records = soup.find_all(class_ ='numero ng-binding')[1].text
    records = re.split(r'\D', str(records))                         
    records = ",".join(records).replace(",", "")
    records = int(records)
    max_page = math.ceil(records / 20)   
    print("----------------")
    print("Scrapuji: " + str(typ_obchodu) + " " + str(typ_stavby))
    print("Celkem inzerátů: " + str(records))
    print("Celkem stránek: " + str(max_page))
    
    if pages == 999:
        print("Scrapuji (pouze) " + str(pages) + " stran.")
        print("----------------")
  
    
    for i in range(pages-1):   
        i = i+2
        
        sys.stdout.write('\r'+ "Strana " + str(i-1) + " = " + str(round(100*(i-1)/(pages), 2)) + "% progress. Zbývá cca: " + str(round(random.uniform(3.4, 3.8)*(pages-(i-1)), 2 )) + " sekund.")    # Asi upravím čas, na rychlejším kabelu v obýváku je to občas i tak 3 sec :O

        url2 = url + "?strana=" + str(i)
        browser.get(url2)

        sleep(random.uniform(1.0, 1.5))

        innerHTML = browser.execute_script("return document.body.innerHTML")
        soup2 = BeautifulSoup(innerHTML,'lxml') 
        
        elements2 = []
        
        for link in soup2.findAll('a', attrs={'href': re.compile("^/detail/prodej/")}):  
            link = link.get('href') 
            elements2.append(link)  
   
        elements2 = elements2[0::2]  
        
        elements = elements + elements2     

    
    browser.quit()   
    
    return elements


In [16]:
def get_img(typ_obchodu = "prodej", typ_stavby = "byty", pages = 1):  
    
    browser = webdriver.Chrome()
    
    
    url_x = r"https://www.sreality.cz/hledani"             
    url = url_x + "/" +  typ_obchodu + "/" +  typ_stavby

    browser.get(url)    # (url).text ??
    sleep(random.uniform(1.0, 1.5))
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,'lxml') # "parser" ?? 

    img = []

    for link in soup.findAll('img', attrs={'src': re.compile("^https://d18")}):      
        link = link.get('src')   
        img.append(link)     
    #del img[0]
    img = img[0::6] 

    
    records = soup.find_all(class_ ='numero ng-binding')[1].text
    records = re.split(r'\D', str(records))                         
    records = ",".join(records).replace(",", "")
    records = int(records)
    max_page = math.ceil(records / 20)   
    print("----------------")
    print("Scrapuji: " + str(typ_obchodu) + " " + str(typ_stavby))
    print("Celkem inzerátů: " + str(records))
    print("Celkem stránek: " + str(max_page))
    
    if pages == 999:
        print("Scrapuji (pouze) " + str(pages) + " stran.")
        print("----------------")
  
    
    for i in range(pages-1):   
        i = i+2
        
        sys.stdout.write('\r'+ "Strana " + str(i-1) + " = " + str(round(100*(i-1)/(pages), 2)) + "% progress. Zbývá cca: " + str(round(random.uniform(3.4, 3.8)*(pages-(i-1)), 2 )) + " sekund.")    # Asi upravím čas, na rychlejším kabelu v obýváku je to občas i tak 3 sec :O

        url2 = url + "?strana=" + str(i)
        browser.get(url2)

        sleep(random.uniform(1.0, 1.5))

        innerHTML = browser.execute_script("return document.body.innerHTML")
        soup2 = BeautifulSoup(innerHTML,'lxml')  

        img2 = []

        for link in soup.findAll('img', attrs={'src': re.compile("^https://d18")}):      
            link = link.get('src')   
            img2.append(link)     
        #del img[0]
        img2 = img2[0::6] 
        img = img + img2

    
    browser.quit()   
    
    return img


In [17]:
def elements_and_ids(x):
    
    elements = pd.DataFrame({"url":x})

    def get_id(x):
        x = x.split("/")[-1]
        return x
    
    len1 = len(elements)
    elements = elements.drop_duplicates(subset = [ "url"], keep = "first", inplace = False)   
    len2 = len(elements)                                                                             
                                                                                                      
    print("-- Vymazáno " + str(len1-len2) + " záznamů kvůli duplikaci.")
    return elements

In [25]:
def scrap_all(typ_obchodu = "prodej", typ_stavby = "byty", pages = 1):
    
    data = get_soup_elements(typ_obchodu = typ_obchodu, typ_stavby = typ_stavby, pages = pages)
    print( "1/2 Data scrapnuta, získávám URLs.")
    
    data = elements_and_ids(data)
    data.to_excel(r"a1_URLs_prodej_byty.xlsx")
    print( "2/2 Získány URL, nyní získávám imgs - několik minut...")
    img1 = get_img(typ_obchodu = typ_obchodu, typ_stavby = typ_stavby, pages = pages)
    l = len(img1)
    data = data.head(l)
    data["img"] = img1
    

    return data

In [26]:
data = scrap_all(pages=10)

----------------
Scrapuji: prodej byty
Celkem inzerátů: 20505
Celkem stránek: 1026
Strana 9 = 90.0% progress. Zbývá cca: 3.58 sekund..1/2 Data scrapnuta, získávám URLs.
-- Vymazáno 496 záznamů kvůli duplikaci.
2/2 Získány URL, nyní získávám imgs - několik minut...
----------------
Scrapuji: prodej byty
Celkem inzerátů: 20397
Celkem stránek: 1020
Strana 9 = 90.0% progress. Zbývá cca: 3.59 sekund..

In [27]:
data[['null','detail', 'prodej','byt','velikost','lokace','id']] = data.url.str.split("/", expand = True)
data = data.drop(['url','null', 'detail','prodej','byt','id'], axis=1)

data.to_csv('sreality.csv', sep=';',index=False)

In [56]:
import psycopg2
conn = psycopg2.connect(host='localhost',
port = '5432',
user = 'postgres',
password = '159357lol',
dbname = 'postgres')

cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS sreality')

create_script = "CREATE TABLE IF NOT EXISTS sreality (IMG VARCHAR(255), Velikost VARCHAR(255), Lokace VARCHAR(255))"
cur.execute(create_script)

with open('sreality.csv', 'r') as f:
    next(f)
    cur.copy_from(f, 'sreality', sep=';', columns=('img', 'velikost', 'lokace'))


conn.commit()
conn.close()
cur.close()

In [57]:
from flask import Flask, render_template
import psycopg2
from IPython.display import display, HTML

app = Flask(__name__)

@app.route('/')
def index():
    conn = psycopg2.connect(host='localhost',
port = '5432',
user = 'postgres',
password = '159357lol',
dbname = 'postgres')
    cursor = conn.cursor()

    cursor.execute("SELECT IMG , Velikost, Lokace FROM sreality LIMIT 500")
    rows = cursor.fetchall()

    # Fetch all rows from the result
    rows = cur.fetchall()

    # Generate HTML for the table
    html = "<html><body><table>"
    html += "<tr><th>Velikost</th><th>Lokace</th><th>Image</th></tr>"

    for row in rows:
        html += "<tr>"
        html += "<td>{}</td>".format(row[0])
        html += "<td>{}</td>".format(row[1])
        html += "<td><img src='{}' width='400' height='300'></td>".format(row[3])
        html += "</tr>"

    html += "</table></body></html>"

    # Close the database connection
    cur.close()
    conn.close()
    
    display(HTML(table))

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2023-07-09 01:25:33,197] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "c:\Users\petre\anaconda3\envs\myenv\lib\site-packages\flask\app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\petre\anaconda3\envs\myenv\lib\site-packages\flask\app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\petre\anaconda3\envs\myenv\lib\site-packages\flask\app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\petre\anaconda3\envs\myenv\lib\site-packages\flask\app.py", line 1469, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\petre\AppData\Local\Temp\ipykernel_9524\1087673341.py", line 20, in index
    rows = cur.fetchall()
psycopg2.InterfaceError: cursor already closed
127.0.0.1 - - [09/Jul/2023 01:25:33] "GET / HTTP/1.1" 50